<a href="https://colab.research.google.com/github/nerdk312/TF_2.0_Keras_VAE_CNN/blob/master/VLAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip uninstall tensorflow
!pip install tensorflow-gpu

Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 380.8MB 45kB/s 
     |████████████████████████████████| 450kB 37.0MB/s 
     |████████████████████████████████| 3.8MB 40.9MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 whi

In [0]:
import os
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [0]:
class conv2d_bn_lrelu(tf.keras.layers.Layer):
  def __init__(self,num_filters,kernel,stride):
    super(conv2d_bn_lrelu,self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(filters=num_filters, kernel_size=kernel, strides=stride, activation=tf.identity)
    self.bn = tf.keras.layers.BatchNormalization()
    self.lrelu = tf.keras.layers.LeakyReLU(alpha=0.1)
  
  def call(self,inputs,training=None):
    conv = self.conv1(inputs)
    conv = self.bn(conv)
    conv = self.lrelu(conv)
    return conv

class conv2d_t_bn_lrelu(tf.keras.layers.Layer):
  def __init__(self,num_filters, kernel,stride):
    super(conv2d_t_bn_lrelu,self).__init__()
    self.conv_t1 = tf.keras.layers.Conv2DTranspose(filters=num_filters,kernel_size=kernel,strides=stride,activation=tf.identity) 
    self.bn = tf.keras.layers.BatchNormalization()
    self.lrelu = tf.keras.layers.LeakyReLU(alpha=0.1)

  def call(self,inputs,training=None):# Call the layers
    convt = self.conv_t1(inputs)
    convt = self.bn(convt)
    convt = self.lrelu(convt)
    return convt

class conv2d_t_bn(tf.keras.layers.Layer):
  def __init__(self,num_filters, kernel,stride):
    super(conv2d_t_bn,self).__init__()
    self.conv_t1 = tf.keras.layers.Conv2DTranspose(filters=num_filters,kernel_size=kernel,strides=stride,activation=tf.identity) 
    self.bn = tf.keras.layers.BatchNormalization()

  def call(self,inputs,training=None):# Call the layers
    convt = self.conv_t1(inputs)
    convt = self.bn(convt)
    return convt

class fc_bn_lrelu(tf.keras.layers.Layer):
  def __init__(self,num_outputs):
    super(fc_bn_lrelu,self).__init__()
    self.fc = tf.keras.layers.Dense(num_outputs, activation=tf.identity)
    self.bn = tf.keras.layers.BatchNormalization()
    self.lrelu = tf.keras.layers.LeakyReLU(alpha=0.1)

  def call(self,inputs,training=None):
    fc1 =  self.fc(inputs)
    fc1 = self.bn(fc1)
    fc1 = self.lrelu(fc1)
    return fc1  

class fc_bn_relu(tf.keras.layers.Layer):
  def __init__(self,num_outputs):
    super(fc_bn_relu,self).__init__()
    self.fc = tf.keras.layers.Dense(num_outputs, activation=tf.identity)
    self.bn = tf.keras.layers.BatchNormalization()
    self.relu = tf.keras.layers.ReLU()

  def call(self,inputs,training=None):
    fc1 =  self.fc(inputs)
    fc1 = self.bn(fc1)
    fc1 = self.relu(fc1)
    return fc1


In [0]:
class Network:
  def __init__(self,dataset,batch_size):
    self.dataset = dataset
    self.batch_size = batch_size

    # These should be updated accordingly
    self.iteration = 0
    self.learning_rate = 0.0
    self.read_only  = False
    
    self.do_generate_samples =  False
    self.do_generate_conditional_samples = False
    self.do_generate_manifold_samples = False
  
  def make_model_path(self):
    if not os.path.isdir("models"):
      os.mkdir("models")
    if not os.path.isdir("models/"+ self.name):
      os.mkdir("models/" + self.name)

# vladder_small

In [0]:
class SmallLayers(tf.keras.layers.Layer):
  def __init__(self, network):
    super(SmallLayers,self).__init__()
    self.network = network

class inference0(SmallLayers): # Nawid - Specifying this layer as a subclass of the other layer so I do not need to keep on writing, self.network
  def __init__(self,network):
    super(inference0,self).__init__(network) # Nawid - This is related to the parameters I need to pass into the parent 
    self.conv2d_bn_lrelu1 = conv2d_bn_lrelu(self.network.cs[1],4,2)
    self.conv2d_bn_lrelu2 = conv2d_bn_lrelu(self.network.cs[2],4,2)
    self.fc_layer = tf.keras.layers.Dense(self.network.cs[3], activation = tf.identity)

  def call(self,input_x,training=None):
    conv1 = self.conv2d_bn_lrelu1(input_x)
    conv2 = self.conv2d_bn_lrelu2(conv1)
    conv2 = tf.reshape(conv2, [-1, np.prod(conv2.get_shape().as_list()[1:])])
    fc1 = self.fc_layer(conv2)
    return fc1

class ladder0(SmallLayers):
  def __init__(self, network):
    super(ladder0,self).__init__(network)
    self.conv2d_bn_lrelu1 = conv2d_bn_lrelu(self.network.cs[1],4,2)
    #print(self.network.cs[1])
    self.conv2d_bn_lrelu2 = conv2d_bn_lrelu(self.network.cs[2],4,2)
    self.fc_mean_layer = tf.keras.layers.Dense(self.network.ladder0_dim, activation = tf.identity)
    self.fc_stddev_layer = tf.keras.layers.Dense(self.network.ladder0_dim, activation = 'sigmoid')

  def call(self,input_x,input_size,ladder_dim,training=None): # Nawid - Added an extra parameter for the shape
    conv1 = self.conv2d_bn_lrelu1(input_x)
    conv2 = self.conv2d_bn_lrelu2(conv1)
    conv2 = tf.reshape(conv2, [-1, np.prod(conv2.get_shape().as_list()[1:])])
    fc1_mean = self.fc_mean_layer(conv2)
    fc1_stddev = self.fc_stddev_layer(conv2)
    # Nawid - Changes to include the loss function in the code
    fc1_stddev += 0.001
    fc1_sample = fc1_mean + tf.multiply(fc1_stddev, tf.keras.backend.random_normal(tf.stack([input_size, ladder_dim]))) 

    # Nawid - Computing the loss term related to this part
    fc1_loss = -tf.math.log(fc1_stddev) + 0.5 * tf.math.square(fc1_stddev) + 0.5* tf.math.square(fc1_mean) - 0.5
    fc1_loss = tf.reduce_mean(tf.reduce_sum(fc1_loss, axis = 1))
    #self.add_loss(fc1_loss)

    return fc1_sample, fc1_loss
    #return fc1_mean, fc1_stddev

class inference1(SmallLayers):
  def __init__(self,network):
    super(inference1,self).__init__(network)
    self.fc_bn_lrelu1 = fc_bn_lrelu(self.network.cs[3])
    self.fc_bn_lrelu2 = fc_bn_lrelu(self.network.cs[3])
    self.fc_output = tf.keras.layers.Dense(self.network.cs[3], activation = tf.identity)

  def call(self,latent1,training=None):
    fc1 = self.fc_bn_lrelu1(latent1)
    fc2 = self.fc_bn_lrelu2(fc1)
    fc3 = self.fc_output(fc2)
    return fc3

class ladder1(SmallLayers):
  def __init__(self,network):
    super(ladder1, self).__init__(network)
    self.fc_bn_lrelu1 = fc_bn_lrelu(self.network.cs[3])
    self.fc_bn_lrelu2 = fc_bn_lrelu(self.network.cs[3])
    self.fc_mean_layer = tf.keras.layers.Dense(self.network.ladder1_dim, activation= tf.identity)
    self.fc_stddev_layer = tf.keras.layers.Dense(self.network.ladder1_dim, activation = 'sigmoid')
    
  def call(self,input_x,input_size,ladder_dim,training=None): # Nawid - input_x is latent1
    fc1 = self.fc_bn_lrelu1(input_x)
    fc2 = self.fc_bn_lrelu2(fc1)
    fc3_mean = self.fc_mean_layer(fc2)
    fc3_stddev = self.fc_stddev_layer(fc2)
    fc3_stddev += 0.001
    fc3_sample = fc3_mean + tf.multiply(fc3_stddev, tf.keras.backend.random_normal(tf.stack([input_size, ladder_dim]))) 

    # Nawid - Computing the loss term related to this part
    fc3_loss = -tf.math.log(fc3_stddev) + 0.5 * tf.math.square(fc3_stddev) + 0.5* tf.math.square(fc3_mean) - 0.5
    fc3_loss = tf.reduce_mean(tf.reduce_sum(fc3_loss, axis = 1))
    #self.add_loss(fc3_loss)
    return fc3_sample, fc3_loss

class ladder2(SmallLayers):
  def __init__(self,network):
    super(ladder2, self).__init__(network)
    self.fc_bn_lrelu1 = fc_bn_lrelu(self.network.cs[3])
    self.fc_bn_lrelu2 = fc_bn_lrelu(self.network.cs[3])
    self.fc_mean_layer = tf.keras.layers.Dense(self.network.ladder2_dim, activation= tf.identity)
    self.fc_stddev_layer = tf.keras.layers.Dense(self.network.ladder2_dim, activation = 'sigmoid')

  def call(self,input_x,input_size,ladder_dim,training=None): # Nawid -  input_x is latent2 
    fc1 = self.fc_bn_lrelu1(input_x)
    fc2 = self.fc_bn_lrelu2(fc1)
    fc3_mean = self.fc_mean_layer(fc2)
    fc3_stddev = self.fc_stddev_layer(fc2)
    fc3_stddev += 0.001
    fc3_sample = fc3_mean + tf.multiply(fc3_stddev, tf.keras.backend.random_normal(tf.stack([input_size, ladder_dim]))) 

    # Nawid - Computing the loss term related to this part
    fc3_loss = -tf.math.log(fc3_stddev) + 0.5 * tf.math.square(fc3_stddev) + 0.5* tf.math.square(fc3_mean) - 0.5
    fc3_loss = tf.reduce_mean(tf.reduce_sum(fc3_loss, axis = 1))
    #self.add_loss(fc3_loss)
    return fc3_sample, fc3_loss

class generative0(SmallLayers):
  def __init__(self, network):
    super(generative0,self).__init__(network)
  
  def combine_noise(self,latent, ladder, method='gated_add', name='default'):
    if method is 'concat':
      return tf.concat(values = [latent,ladder], axis = len(latent.get_shape())-1)
    else:
      if method is 'add':
        return latent + ladder
      elif method is 'gated_add':
        return latent + tf.multiply(self.gate, ladder)

  def build(self,input_shape):    
    self.gate = tf.Variable(tf.keras.backend.random_normal(shape=(input_shape[1:]))) # NAWID -input shape includes the batch, therefore I need to use values from past the zero element:
    self.ladder0_present_fc_bn_lrelu = fc_bn_lrelu(self.network.cs[3])
    self.fc_bn_relu1 = fc_bn_relu(int(self.network.fs[2]*self.network.fs[2]*self.network.cs[2]))
    self.conv2d_t_bn_lrelu1 = conv2d_t_bn_lrelu(self.network.cs[1],2,2)
    self.output_conv_t = tf.keras.layers.Conv2DTranspose(self.network.data_dims[-1],2,2, activation= 'sigmoid')
    #self.conv2d_t_bn_lrelu1 = conv2d_t_bn_lrelu(self.network.cs[1],4,2, is_training)
#    self.output_conv_t = tf.keras.layers.Conv2DTranspose(self.network.data_dims[-1],4,2, activation= 'sigmoid')

  def call(self,input_x, ladder0,training=None): # Nawid  - Input_x corresponds to latent1    
    ladder0 = self.ladder0_present_fc_bn_lrelu(ladder0)
    latent1 = self.combine_noise(input_x, ladder0, name="generative0")

    fc1 = self.fc_bn_relu1(latent1)
    #print("Before reshaped:",fc1.shape)
    fc1 = tf.reshape(fc1, tf.stack([tf.shape(fc1)[0], self.network.fs[2], self.network.fs[2], self.network.cs[2]]))
    #print("Reshaped fc1", fc1.shape)
    conv1 = self.conv2d_t_bn_lrelu1(fc1)
    #print("Conv1 shape",conv1.shape)
    output = self.output_conv_t(conv1)
    #print("Output shape:",output.shape)
    output = (self.network.dataset.range[1] - self.network.dataset.range[0])* output + self.network.dataset.range[0]
    #print("final output shape:", output.shape)
    '''
    # Loss and training operators
    reconstruction_loss =  tf.reduce_mean(tf.abs(output - original_input))
    reg_coeff = 1.0
    reconstruction_loss *= loss_ratio * np.prod(data_dims)
    overall_loss = reg_coeff * regularisation + reconstruction_loss
    self.add_loss(overall_loss)
    '''
    return output

 
class generative1(SmallLayers):
  def __init__(self, network):
    super(generative1,self).__init__(network)

  def combine_noise(self,latent, ladder, method='gated_add', name='default'):
    if method is 'concat':
      return tf.concat(values = [latent,ladder], axis = len(latent.get_shape())-1)
    else:
      if method is 'add':
        return latent + ladder
      elif method is 'gated_add':
        return latent + tf.multiply(self.gate, ladder)

  def build(self,input_shape):
    self.gate = tf.Variable(tf.keras.backend.random_normal(shape=(input_shape[1:]))) # NAWID -input shape includes the batch, therefore I need to use values from past the zero element:    
    self.ladder1_present_fc_bn_relu = fc_bn_relu(self.network.cs[3])
    self.fc_bn_relu1 = fc_bn_relu(self.network.cs[3])
    self.fc_bn_relu2 = fc_bn_relu(self.network.cs[3])
    self.output_fc = tf.keras.layers.Dense(self.network.cs[3], activation= tf.identity)

  def call(self,input_x,ladder1,training=None): # Nawid  - Input_x corresponds to latent1    
    ladder1 = self.ladder1_present_fc_bn_relu(ladder1)
    latent2 = self.combine_noise(input_x, ladder1, name="generative1")
    fc1 = self.fc_bn_relu1(latent2)
    fc2 = self.fc_bn_relu2(fc1)
    fc3 = self.output_fc(fc2)
    return fc3

class generative2(SmallLayers):
  def __init__(self, network):
    super(generative2,self).__init__(network)
    self.fc_layer1 = fc_bn_relu(self.network.cs[3])
    self.fc_layer2 = fc_bn_relu(self.network.cs[3])
    self.fc_layer3 = tf.keras.layers.Dense(self.network.cs[3])#self.network.cs[3])
  
  def call(self,input_x,training=None): # Nawid -  In this case ladder 2 is in the input   
    fc1 = self.fc_layer1(input_x)
    fc2 = self.fc_layer2(fc1)
    fc3 = self.fc_layer3(fc2)
    return fc3

In [0]:
class VLadder(Network):
  def __init__(self, dataset, name=None, reg='kl', batch_size = 100, restart= False):
    super(VLadder,self).__init__(dataset, batch_size)
    #Network.__init__(self, dataset, batch_size)
    #print(batch_size)
    if name is None or name == '':
      self.name = "vladder_%s" % dataset.name # Nawid - GIves a name based on the dataset
    else:
      self.name = name

    self.dataset = dataset
    self.batch_size = batch_size
    self.data_dims = self.dataset.data_dims 

    self.latent_noise = False
    
    self.fs = [self.data_dims[0], self.data_dims[0]//2, self.data_dims[0]//4 , self.data_dims[0]//8, self.data_dims[0]//16]
    self.reg = reg
    
    if self.reg != 'kl':
      print(' Unknown regularization, supported: kl')

    if self.name == "vladder_mnist":
      self.cs = [1, 64, 128, 1024] # Nawid-  Different number of cs and lower ladder dimension
      self.ladder0_dim = 2
      self.ladder1_dim = 2
      self.ladder2_dim = 2
      self.num_layers = 3
      loss_ratio = 8.0
      self.error_scale = 8.0
      layers = SmallLayers(self) # Nawid- Uses small layers 
    else:
      print('Unknown architecture name %s' % self.name)
      exit(-1)
  
    self.self = self
    '''
    self_input_placeholder = self.dataset.train_data
    input_size = self_input_placeholder.shape[0]
    print(input_size)
    '''
    self_input_placeholder = tf.keras.layers.Input(shape= self.data_dims, dtype=tf.float32)  # Nawid - Input layer
    input_size = tf.shape(self_input_placeholder)[0]
    print('input_size1:',input_size)

    # Define the inference network
    self.regularisation = 0.0

    if self.ladder0_dim>0:
      ladder0_layer = ladder0(self) # Nawid - Self in this case is the network
      
      self.iladder0_sample, iladder0_loss = ladder0_layer(self_input_placeholder,input_size, self.ladder0_dim)
      self.regularisation += iladder0_loss
    
    if self.num_layers >= 2:
      inference0_layer = inference0(self)
      self.ilatent1_hidden = inference0_layer(self_input_placeholder)      
      if self.ladder1_dim >0:
        ladder1_layer = ladder1(self)
        self.iladder1_sample, iladder1_loss = ladder1_layer(self.ilatent1_hidden,input_size,self.ladder1_dim)
        self.regularisation += iladder1_loss
    
    if self.num_layers >= 3:
      inference1_layer = inference1(self)
      self.ilatent2_hidden = inference1_layer(self.ilatent1_hidden)
      if self.ladder2_dim >0:
        ladder2_layer = ladder2(self)
        self.iladder2_sample, iladder2_loss = ladder2_layer(self.ilatent2_hidden,input_size,self.ladder2_dim)
        self.regularisation += iladder2_loss
    
    # Define the generative network
#    self.ladders = {}
    if self.num_layers >= 3 and self.ladder2_dim >0:
      generative2_layer = generative2(self)
      tlatent2_state = generative2_layer(self.iladder2_sample)
          
    if self.num_layers >= 2 and self.ladder1_dim >0:
      generative1_layer = generative1(self)
      tlatent1_state = generative1_layer(tlatent2_state, self.iladder1_sample)
    

    

    if self.ladder0_dim > 0:
      generative0_layer = generative0(self)
      self.toutput = generative0_layer(tlatent1_state, self.iladder0_sample)
    '''
    #def call(self,input_x, ladder0, original_input,loss_ratio,data_dims,regularisation): # Nawid  - Input_x corresponds to latent1
    # Loss and training operators

    self.reconstruction_loss =  tf.reduce_mean(tf.abs(self.toutput - self_input_placeholder))
    self.reg_coeff = 1.0

    
    if self.reg == 'kl':
      self.reconstruction_loss *= loss_ratio * np.prod(self.data_dims)
      self.loss = self.reg_coeff * self.regularization + self.reconstruction_loss
    '''    

    self.model = tf.keras.Model(self_input_placeholder,self.toutput)

    self.model.summary()
    tf.keras.utils.plot_model(self.model, 'my_new_model.png')
    

class Dataset:
  def __init__(self):
    self.name = "abstract"
    self.data_dims = []
    self.width = -1
    self.height = -1
    self.train_size = -1
    self.test_size = -1
    self.range = [0.0, 1.0]

class MnistDataset(Dataset):
  def __init__(self):
    super(MnistDataset,self).__init__()
    (self.train_data, self.train_label), (self.test_data, self.test_label) = tf.keras.datasets.mnist.load_data()
    self.train_data = self.train_data.astype('float32') #/ 255
    self.train_data = self.train_data.reshape([-1, 28,28,1])
    #self.train_data = self.train_data[0:5]
    #print(self.train_data.shape)

    self.test_data = self.test_data.astype('float32')#/255
    self.test_data = self.test_data.reshape([-1, 28,28,1])

    self.name = "mnist"
    self.data_dims = [28,28,1]

#    self.train_size = 50000
#    self.test_size = 10000
    self.range = [0.0,1.0]

dataset = MnistDataset()
train_data = dataset.train_data
ladder_network = VLadder(dataset,batch_size=100) # Nawid -Need to define it manually so that the value does not change.
VLAE =ladder_network.model
regularisation = ladder_network.regularisation
print(regularisation)
#output = VLAE.predict(train_data)


11493376/11490434 [==============================] - 0s 0us/step
input_size1: Tensor("strided_slice:0", shape=(), dtype=int32)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
inference0 (inference0)         (None, 1024)         3410880     input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Shape (TensorFlowOp [(4,)]               0           input_1[0][0]                    
__________________________________________________________________________________________________
inference1 (inference1)         (None, 1024)         3156992     i

In [0]:
loss_ratio = 8.0
data_dims = [28,28,1]
batch_size = 100 

In [0]:
def VLAE_loss(x,reconstructed_x):
  reconstruction_loss =  tf.reduce_mean(tf.abs(reconstructed_x - x))
  reg_coeff = 1.0
  reconstruction_loss *= loss_ratio * np.prod(data_dims)
  #overall_loss = reconstruction_loss
  overall_loss = reg_coeff * regularisation + reconstruction_loss
  print(regularisation) # Nawid- Print to show that this is changing each time
  return overall_loss

In [0]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, period = 5) # Nawid- Used to save every 5 epochs - Instance of the model checkpoint callback

VLAE.compile(optimizer='adam',loss = VLAE_loss, experimental_run_tf_function=False)
VLAE.fit(train_data, train_data,
        shuffle=True,
        epochs=10,
        batch_size=100,
        callbacks=[cp_callback])  # Pass callback to training))
        

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir) # Nawid - Obtains the latest value of weights
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)



In [0]:
decoder_imgs = VLAE.predict(train_data)
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(10):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_train[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()